# My College Basketball Data

In [10]:
#import
import csv
import numpy as np
import torch 

In [11]:
#set up dictionary for team number and team name
teamidentities = {}
with open('Teams.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    
    for row in readCSV:
        teamidentities.update( {row[0]: row[1]})
    
    #deletes first key 
    del teamidentities["Team_Id"]

In [12]:
#read and data split tourney seeds
#1985-2017

trainingSetSeeds = []
testSetSeeds = []
validationSetSeeds = []

with open('TourneySeeds.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
   
    for row in readCSV:
        if row[0] == "2016":
            testSetSeeds.append(row)
        elif row[0] == "2017":
            validationSetSeeds.append(row)
        else:
            trainingSetSeeds.append(row)

In [13]:
#read and data split regular season stats
#2003-2015
#trainingSet is 2003-2013
#testSet is 2014
#validationSet 2015

trainingSet = []
testSet = []
validationSet = []

def dataSplit():
    for row in readCSV:
        if row[0] == "2014":
            testSet.append(row)
        elif row[0] == "2015":
            validationSet.append(row)
        else:
            trainingSet.append(row)

with open('RegularSeasonDetailedResults.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    
    dataSplit()

In [14]:
#read and data split tournament stats
#2003-2016

remainingDataSet = []
with open('TourneyDetailedResults.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    
    dataSplit()

#used dataSplit() to assemble trainingDataSet, testDataSet, validationDataSet by combining data from RegularSeasonedDetailedResults.csv and TourneyDetailedResults
#trainingDataSet made up of data from RSDR years of 2003-2013 and TDR years 2003-2013, 2016
#testDataSet made up of data from RSDR years 2014 and TDR years 2014
#validationDataSet made up of data from RSDR years 2015 and TDR years 2015

#delete these lists because they are strings not numbers
del trainingSet[0]
del trainingSet[65920]

In [15]:
#extracting features for each DataSet
#first half of the feature vector is winning team stats and last half is losing team stats
#features taken out: [Wteam, Wfgm, Wfga, Wfgm3, Wfga3, Wftm, Wfta, Wor, Wdr, Wast, Wto, Wstl, Wblk, Wpf, Lteam, Lfgm, Lfga, Lfgm3, Lfga3, Lftm, Lfta, Lor, Ldr, Last, Lto, Lstl, Lblk, Lpf]
#score is not taken into account

#function to featureExtract
def featureExtract(dataSetArray):
    featureExtractArray = []
    for i, row in enumerate(dataSetArray):
        test = [row[2], row[8], row[9], row[10], row[11], row[12], row[13], row[14], row[15], row[16], row[17], row[18], row[19], row[20], row[4], row[21], row[22], row[23], row[24], row[25], row[26], row[27], row[28], row[29], row[30], row[31], row[32], row[33]]
        
        try:
        
            test = [float(ele) for ele in test]
        except:
            print(i, test)
        featureExtractArray.append(test)
    return featureExtractArray[1:].copy()
    #print(featureExtractArray)
    
#trainingSet feature extraction
featurestrainingSet = featureExtract(trainingSet)

#testSet feature extraction
featurestestSet = featureExtract(testSet)

#validationSet feature extraction
featuresvalidationSet = featureExtract(validationSet)

In [16]:
#making labels

#randomization of winning and losing team statistics from feature trainingDataSet
labelstrainingSet = np.random.randint(2, size = len(featurestrainingSet))
#28 elements in featuretrainingDataSet

# randomization of winning and losing team statistics from feature testDataSet
labelstestSet = np.random.randint(2, size = len(featurestestSet))

#randomization of winning and losing team statistics from feature validationDataSet
labelsvalidationSet = np.random.randint(2, size = len(featuresvalidationSet))

n_feat = len(featurestrainingSet[0])
assert n_feat % 2 == 0

#randomization function flips the back half of the data if corresponding label is a 1
def randomization(labelList, originalDataSet):
    randomizedDataSet = []
    for i, do_flip in enumerate(labelList):    
        #if its a 1 that means the back half won so flip
        x = originalDataSet[i] 
        if do_flip == 1:   
            x_rand = x[n_feat // 2:] + x[0:n_feat // 2]
        else:
            x_rand = x
        randomizedDataSet.append(x_rand)
    return randomizedDataSet

randomizedfeaturetrainingSet = randomization(labelstrainingSet, featurestrainingSet)
randomizedfeaturetestSet = randomization(labelstestSet, featurestestSet)
randomizedfeaturevalidationSet = randomization(labelsvalidationSet, featuresvalidationSet)

#test case
print(randomizedfeaturetrainingSet[0])
print(labelstrainingSet[0])

[1393.0, 24.0, 67.0, 6.0, 24.0, 9.0, 20.0, 20.0, 25.0, 7.0, 12.0, 8.0, 6.0, 16.0, 1272.0, 26.0, 62.0, 8.0, 20.0, 10.0, 19.0, 15.0, 28.0, 16.0, 13.0, 4.0, 4.0, 18.0]
1


In [17]:
#pull labels out and then flip the respecitve lists within the data set corresponding to the data set
#save the array into np and then turn into torch
#implement logistical reggression

In [21]:
#converts to numpy array and pytorch array
def PytorchArray (labels, randomizedDataSet):
    randomizedDataSet_np = np.array(randomizedDataSet).astype(np.float32)
    labels_np = np.array(labels)
    randomizedDataSet_pytorch = torch.from_numpy(randomizedDataSet_np)
    labels_pytorch = torch.from_numpy(labels_np)    
    return randomizedDataSet_pytorch, labels_pytorch

#assigns variables names to pytorch array
featurestrainingData_pytorch, labelstrainingData_pytorch = PytorchArray(labelstrainingSet, randomizedfeaturetrainingSet)
featurestestData_pytorch, labelstestData_pytorch = PytorchArray(labelstestSet, randomizedfeaturetestSet)
featuresvalidationData_pytorch, labelsvalidationData_pytorch = PytorchArray(labelsvalidationSet, randomizedfeaturevalidationSet)

#save data sets pytorch array
torch.save(featurestrainingData_pytorch, 'featurestraining_pytorch.pt')
torch.save(featurestestData_pytorch, 'featurestest_pytorch.pt')
torch.save(featuresvalidationData_pytorch, 'featuresvalidation_pytorch.pt')

#save labels pytorch array
torch.save(labelstrainingData_pytorch, 'labelstraining_pytorch.pt')
torch.save(labelstestData_pytorch, 'labelstest_pytorch.pt')
torch.save(labelsvalidationData_pytorch, 'labelsvalidation_pytorch.pt')